Experiment 1: 

Run HMMM on all users 

In [1]:
import hmm_scaled
import os 

import pandas as pd 
import seaborn as sns 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

import warnings
warnings.filterwarnings("error", message=".*check_inverse*.",
                        category=UserWarning, append=False)

In [8]:
def generate_dic(items): 
    """ Genderates a dictionay for all values in item """
    new_dic = {}
    count = 0
    for unique in items: 
        new_dic[unique]  = count
        count +=1
    return new_dic

def column_scaler(data): 
    """
    scales each column of data 
    """
    scaler = StandardScaler()
    for value in data.columns: 
        data_sub_ = pd.DataFrame(data.loc[:,value])
        data.loc[:,value] =scaler.fit_transform(data_sub_)
    return data

def kmean_prop(data, column): 
    data_value = data.groupby(column).agg(count = pd.NamedAgg(column= column, aggfunc= 'count'))
    percentage = []
    for i in range(0, len(data_value)): 
        result = round(((data_value.iloc[i,0] / len(data)) * 100),2)
         
        percentage.append(result)
    data_value['percentage (%)'] = percentage
        
    return data_value

def PCA_Col_names(count):
    """
    Remove columns in a PCA that meets the threshold count 
    
    """
    colname = []
    rowname = 1
    while rowname <= count: 
        colname.append(f'Principle Component {rowname}')
        rowname += 1 
    return (colname)

class page_description:
    """
    Class to analyze direction data. Purpose is to determine the movement count thresholdhold for hmm model.  
    """
    def __init__(self, data,page_number):
        self.page_number = page_number
        self.page = data[data['Page count'] == f'page {self.page_number}']
        self.data = data 

    def page_segment(self):
        """
        Desplays the user id belong to a that segment
        """
        page = self.data[self.data['Page count'] == f'page {self.page_number}']
        page = page.groupby('User Id').count()[['Page count']]
        return page

    def page_description(self):
        """
        Provide summary statistics on a page 
        """
        
        return self.page_segment().describe()

    def percentage(self):
        """
        prints the percentage of the total number of counts recorded on this page
        """
        
        value = f' {round((len(self.page) / len(self.data)) * 100,2)}% of all the mouse movemens are recorded in page {self.page_number}'
        
        return value

In [3]:
# set working directory (Change for youe)
os.chdir('C:/Users/ander/Google Drive/Columbia/Fall 2019/Capstone/Dotin-Columbia-Castone-Team-Alpha-')
data = pd.read_csv('Models/Q1_Mouse Activity/Data/direction_data.csv')
data.head()

,Unnamed: 0,User Id,Distance X,Distance Y,Direction,Page count,observation
0,0,371,707,224.0,North East,page 1,1
1,1,371,4,-20.0,North West,page 1,2
2,2,371,0,-30.0,South,page 1,3
3,3,371,0,-18.0,South,page 1,4
4,4,371,1,-2.0,North West,page 1,5


In [10]:
page_number_list =[]
for value in range(1,16):
    page_ = f'page {value}'
    page_number_list.append(page_)
    

# limit page observation per page 
final_data = data[0:0]

for page in range(0,len(page_number_list)):
    observation_limit = round(page_description(data, page +1).page_description().iloc[4]['Page count'])
    page_subset =  data[data['Page count'] == page_number_list[page]]
    page_shrunk = page_subset[page_subset.observation <= observation_limit]
    page_ = page_shrunk.groupby('User Id').count()['observation']
    user_list = page_[page_  == observation_limit].index
    updated_dat = page_shrunk[page_shrunk['User Id'].isin(user_list)]
    final_data = pd.concat([final_data,updated_dat])    

In [11]:
for page in range(0,15):
    pages___ = final_data[final_data['Page count']== page_number_list[page]]
    user_count = len(pages___.groupby('User Id').count()['observation'])
    print(f' Page {page+1}: {user_count}')

 Page 1: 136
 Page 2: 161
 Page 3: 159
 Page 4: 157
 Page 5: 152
 Page 6: 146
 Page 7: 154
 Page 8: 158
 Page 9: 148
 Page 10: 150
 Page 11: 140
 Page 12: 136
 Page 13: 128
 Page 14: 110
 Page 15: 99


In [12]:
user_direction = data[['User Id','Page count', 'Direction']]
flatten_data = user_direction.groupby(['User Id','Page count'], as_index=False).agg(list)

user_dict = generate_dic(set(data['Direction']))

direction_numerized =[]
for row in range(0,len(flatten_data)): 
    value = flatten_data.iloc[row,2]
    numerized =[user_dict.get(key, value) for key in value]
    direction_numerized.append(numerized)
flatten_data['Direction Numerized'] = direction_numerized

page_number_list =[]
for value in range(1,16):
    page_ = f'page {value}'
    page_number_list.append(page_)

In [13]:
unique_user = set(data['User Id'])
unique_page = set(data['Page count'])
user_fraud_matrix = pd.DataFrame(data=0 , index=unique_user, columns=unique_page)
user_fraud_matrix

,page 13,page 4,page 7,page 6,page 12,page 14,page 3,page 11,page 8,page 15,page 5,page 10,page 2,page 9,page 1
371,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
374,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
376,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1835,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1836,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
hmm= hmm_scaled.HMM(len(user_dict))

In [ ]:
for count in range(0,len(page_number_list)):
    page_ = flatten_data[flatten_data['Page count'] == page_number_list[count]]
    page_= page_.set_index('User Id')
    hmm.fit(list(page_['Direction Numerized']))
    for index in user_fraud_matrix.index:
        try: 
            user_fraud_matrix.loc[index, page_number_list[count]] = hmm.log_likelihood(page_.loc[index, 'Direction Numerized'])
        except KeyError:
            continue

iteration: 0
iteration: 10
iteration: 20
Fit duration: 14:20:42.491223
iteration: 0
iteration: 10
iteration: 20
Fit duration: 0:37:27.705832
iteration: 0


### KMeans

In [ ]:
user_fraud_matrix_scaled = column_scaler(user_fraud_matrix)

In [ ]:
fig = plt.figure(figsize=(16,10))
corr = user_fraud_matrix_scaled.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True)
plt.title('Probability correlation by page')
plt.show()

In [ ]:
# calculate distortion for a range of number of cluster
distortions = []
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(user_fraud_matrix_scaled)
    distortions.append(km.inertia_)

# plot
fig = plt.figure(figsize=(16,10))
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.tight_layout()
plt.show()

In [ ]:
km_1 = KMeans(
    n_clusters=8, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)

user_fraud_matrix_scaled['K_means']=  km_1.fit_predict(user_fraud_matrix_scaled)
kmean_prop(user_fraud_matrix_scaled, 'K_means')

In [ ]:
sns.pairplot(user_fraud_matrix_scaled.iloc[:,[1,2,3,4,5,6,7,15]], hue = 'K_means', palette='muted')

In [ ]:
user_fraud_matrix_scaled = user_fraud_matrix_scaled.drop(columns=['K_means'])

In [ ]:
pca = PCA()
pca_fit = pca.fit_transform(user_fraud_matrix_scaled)  

explained_ratio = pca.explained_variance_ratio_
explained_ratio = np.round(explained_ratio, 2)
np.cumsum(explained_ratio)

In [ ]:
plt.plot(np.cumsum(explained_ratio))
plt.title('Cumulative sum of explained ratio')
plt.xlabel('Number of features')
plt.ylabel('Percentae of variance captures')
plt.tight_layout()
plt.show()

In [ ]:
pca_col_names = PCA_Col_names(10)
pca = PCA(n_components=10)
pca_fit = pca.fit_transform(user_fraud_matrix_scaled)
pca_dat = pd.DataFrame(pca_fit,columns=pca_col_names)

In [ ]:
# calculate distortion for a range of number of cluster
distortions = []
for i in range(1, 11):
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(pca_dat)
    distortions.append(km.inertia_)

# plot
fig = plt.figure(figsize=(16,10))
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.tight_layout()
plt.show()

In [ ]:
km_1 = KMeans(
    n_clusters=8, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)

pca_dat['K_means']=  km_1.fit_predict(pca_dat)
kmean_prop(pca_dat, 'K_means')

In [ ]:
sns.pairplot(pca_dat.iloc[:,[1,2,3,4,5,6,7,10]], hue = 'K_means', palette='muted')

In [ ]:
# Compute DBSCAN on scaled data 
scaled_db = DBSCAN(eps=0.40, min_samples=5).fit(user_fraud_matrix_scaled)
scaled_db_labels = scaled_db.labels_
len(set(scaled_db_labels))

In [ ]:
scaled_cluster, count = np.unique(scaled_db_labels, return_counts=True)
dict(zip(scaled_cluster, count))

In [ ]:
# consolidate outlier 
new_labels = []
for i in scaled_db_labels: 
    if i == -1: 
        new_labels.append('Normal')
    else: 
        new_labels.append('Outlier')

np.unique(new_labels, return_counts=True)        

In [ ]:
user_fraud_matrix_scaled['DB Scan Label'] = user_fraud_matrix_scaled['DB Scan Label'] = new_labels 

In [ ]:
sns.pairplot(user_fraud_matrix_scaled.iloc[:,[1,2,3,4,5,6,7,15]], hue = 'DB Scan Label', palette='muted')